In [2]:
import numpy as np
import re
import requests
import pandas as pd


In [3]:


url = 'http://localhost:9000/stored_tweets'
data = requests.get(url).json()

print(data)



{'fileName': 'data\\test-tweets\\smol\\10.jsonl', 'tweets': [{'tweet_id': 1133705772246786049, 'tweet_text': 'Waking to smoky sunrise in Calgary today   eery, but beautiful yycweather sunrise abfire ', 'features': {'numb_of_urls': None, 'numb_of_hashtags': 3.0, 'numb_of_personal_pronouns': 0.0, 'numb_of_present_tenses': 0.0, 'weighted_length': 115.0, 'permillage': 410.0, 'tweet_created_at': 1559131493000.0, 'tweet_id_str': 1.133705772246786e+18, 'positive_sentiment': 0.0, 'negative_sentiment': 0.0, 'numb_of_mentions': 0.0, 'numb_of_media': 1.0, 'numb_of_past_tenses': 0.0, 'numb_of_weird_chars': 16.0, 'numb_of_questions': 0.0, 'numb_of_emoticons': 0.0, 'numb_of_swearing_word': 0.0, 'numb_of_slang_words': 0.0, 'numb_of_intensifiers': 0.0, 'tweet_length': 89.0, 'userFollowersCount': 15.0, 'userFriendsCount': 81.0, 'user_numb_of_tweets': 0.0, 'user_list_count': 0.0, 'dict_precision': 0.0, 'dict_recall': 0.0, 'dict_f_measure': 0.0, 'offset': 0.0, 'is_verified': 0.0}, 'sentiment': 'Type = de

In [4]:
df = pd.DataFrame(data)

df = pd.json_normalize(df['tweets'])
df


,tweet_id,tweet_text,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,...,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified
0,1133705772246786049,Waking to smoky sunrise in Calgary today eer...,"Type = dense , numRows = 25 , numCols = 1\r,-0...","{1.7785985,3.1451268,-5.3303847,0.53038704,3.8...",None,3.0,0.0,0.0,115.0,410.0,...,89.0,15.0,81.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1134018172309295104,BREAKING Hundreds of evacuees in Calling Lake...,"Type = dense , numRows = 25 , numCols = 1\r,-0...","{4.7370405,2.1589317,-2.065543,-3.1837502,-3.8...",None,2.0,0.0,0.0,133.0,475.0,...,107.0,865.0,50.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0
2,1131561862343086081,High Level wildfire crews fight fire with fire...,"Type = dense , numRows = 25 , numCols = 1\r,-0...","{2.97405,-0.41674098,1.8703463,-2.558454,-1.56...",None,0.0,0.0,0.0,94.0,335.0,...,71.0,1295.0,1348.0,0.0,57.0,0.0,0.0,0.0,0.0,1.0


In [5]:
#df = pd.concat([df['embeddings'], df[1].str.split(', ', expand=True)], axis=1)

df['sentiment'] = df[df['sentiment'].apply(lambda x:pd.Series(x))]

In [6]:
#df['embeddings'] = df['embeddings'].str.extract('(\d+)', expand=False)
df['embeddings'] = df['embeddings'].str[1:]
df_embeddings = pd.concat([df['embeddings'].str.split(',', expand=True)], axis=1, keys="e")

df.join(df_embeddings)

/Users/mark/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,tweet_id,tweet_text,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,...,"(e, 41)","(e, 42)","(e, 43)","(e, 44)","(e, 45)","(e, 46)","(e, 47)","(e, 48)","(e, 49)","(e, 50)"
0,1133705772246786049,Waking to smoky sunrise in Calgary today eer...,NaN,"1.7785985,3.1451268,-5.3303847,0.53038704,3.84...",None,3.0,0.0,0.0,115.0,410.0,...,0.13766381,-2.03385,-0.59648,-2.563134,3.8937702,0.37940502,-5.230379,-0.48351637,0.18215394,}
1,1134018172309295104,BREAKING Hundreds of evacuees in Calling Lake...,NaN,"4.7370405,2.1589317,-2.065543,-3.1837502,-3.86...",None,2.0,0.0,0.0,133.0,475.0,...,2.1835423,2.3010478,-1.2485831,-2.7621722,1.2965181,-5.134682,-2.6551764,-2.0404568,-4.251936,}
2,1131561862343086081,High Level wildfire crews fight fire with fire...,NaN,"2.97405,-0.41674098,1.8703463,-2.558454,-1.569...",None,0.0,0.0,0.0,94.0,335.0,...,0.16428995,2.4351277,-1.8204299,1.3995962,1.908064,-1.961153,-3.1902032,0.07204503,-2.7900722,}


In [7]:
cols = ['Embeddings']
df = pd.concat([df['embeddings'].str.split(',', expand=True) for x in cols], axis=1, keys=df.columns)
df.columns = df.columns.map(lambda x: '_'.join((x[0], str(x[1]))))
#df = df.replace({'':np.nan, None:np.nan})
df2 = pd.concat([df1, df.drop(cols, axis=1)],axis=1)
print (df2)
print (df)

NameError: name 'df1' is not defined

In [97]:
df

,tweet_id_0,tweet_id_1,tweet_id_2,tweet_id_3,tweet_id_4,tweet_id_5,tweet_id_6,tweet_id_7,tweet_id_8,tweet_id_9,...,tweet_id_41,tweet_id_42,tweet_id_43,tweet_id_44,tweet_id_45,tweet_id_46,tweet_id_47,tweet_id_48,tweet_id_49,tweet_id_50
0,{1.7785985,3.1451268,-5.3303847,0.53038704,3.8427823,-2.587076,-5.077908,0.515931,-0.96028394,1.315377,...,0.13766381,-2.03385,-0.59648,-2.563134,3.8937702,0.37940502,-5.230379,-0.48351637,0.18215394,}
1,{4.7370405,2.1589317,-2.065543,-3.1837502,-3.8683202,0.47769997,-6.617863,3.462947,3.8819861,-3.5452929,...,2.1835423,2.3010478,-1.2485831,-2.7621722,1.2965181,-5.134682,-2.6551764,-2.0404568,-4.251936,}
2,{2.97405,-0.41674098,1.8703463,-2.558454,-1.5693818,-0.615715,-5.20854,1.8524354,2.39962,-5.125714,...,0.16428995,2.4351277,-1.8204299,1.3995962,1.908064,-1.961153,-3.1902032,0.07204503,-2.7900722,}
